In [41]:
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import keras
import sklearn.metrics
import keras.datasets.mnist

In [2]:
# Funciones útiles
def plot_loss(history, metric):
  plt.plot(history.history[metric], label='Training data')
  plt.plot(history.history['val_' + metric], label='Validation data')
  plt.title(metric)
  plt.ylabel(metric)
  plt.xlabel('Epoch')
  plt.legend(loc="upper right")
  plt.show()

In [39]:
# Archivos donde se guardan los modelos de alta precisión de de baja precisión

filename_modelo_heavy = "./model_heavy.keras"
filename_modelo_lite = "./model_lite.keras"

In [28]:
# Carga del dataset

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print(x_train.shape)
cantidad_imagenes = x_train.shape[0]

# Escalado a [0,1]
x_train = x_train / 255
x_test = x_test / 255

x_test = x_test.astype(np.float32)

In [ ]:
# Visualización de una imagen cualquiera
azar = random.randint(0, cantidad_imagenes)
imagen = x_train[azar,:,:]
print("Clase " + str(y_train[azar]))
plt.rcParams['image.cmap'] = 'gray' # para dibujarla en tonos de grises
plt.figure(figsize = (1, 1))
plt.imshow(imagen)

In [3]:
# Armar la arquitectura

# Las capas convolucionales esperan "tensores" en la entrada
# Por más que sean imágenes en tonos de grises hay que agregarle la tercer dimensión,
# los canales, que en este caso es solo uno

x_train = x_train.reshape(cantidad_imagenes, x_train.shape[1], x_train.shape[2], 1)

d_in = x_train[0].shape
d_out = 10

modelo = keras.Sequential([
    keras.layers.Conv2D(64, input_shape=d_in, kernel_size=3, activation="relu"),
    keras.layers.Flatten(),
    keras.layers.Dense(d_out, activation="softmax")
	])

modelo.compile(
  optimizer = keras.optimizers.SGD(learning_rate=0.001),
  loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']
)

In [38]:
# Entrenar el modelo

epochs = 10
batch_size = 64

history = modelo.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=True, validation_split=0.2)

plot_loss(history, "loss")
plot_loss(history, "accuracy")

# Guardar modelo en alta precisión
modelo.save(filename_modelo_heavy)

In [ ]:
# Validación con conjunto de test

results = modelo.predict(x_test)
preds = np.argmax(results, axis=1)
sklearn.metrics.ConfusionMatrixDisplay.from_predictions(y_test, preds)

In [40]:
# Cuantificar el modelo

converter = tf.lite.TFLiteConverter.from_keras_model(modelo)
tflite_model = converter.convert()

# Guardar modelo en baja precisión
with open(filename_modelo_lite, 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Comparar tamaños

sizefile_heavy = os.stat(filename_modelo_heavy).st_size
sizefile_lite = os.stat(filename_modelo_lite).st_size
print("Tamaño del modelo de alta precisión: " + str(sizefile_heavy) + " bytes")
print("Tamaño del modelo de baja precisión: " + str(sizefile_lite) + " bytes")

In [30]:
# Cargar el modelo
# Esto es lo que debería hacerse en el dispositivo móvil o microcrontolador

(muestras, ancho, alto) = x_test.shape

interpreter = tf.lite.Interpreter(model_path=filename_modelo_lite)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
input_shape = input_details[0]['shape']
input_index = input_details[0]['index']

output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

In [ ]:
# Hacer la inferencia de las muestras
# Esto es lo que debería hacerse en el dispositivo móvil o microcrontolador

preds = []
for m in range(muestras):
  input_data = x_test[m].reshape(1, ancho, alto, 1)
  interpreter.set_tensor(input_index, input_data)

  interpreter.invoke()

  output_data = interpreter.get_tensor(output_index)

  clase = np.argmax(output_data)
  preds.append(clase)

sklearn.metrics.ConfusionMatrixDisplay.from_predictions(y_test, preds)